### Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

Задание:

- Используя Spark прочитайте данные из файла csv.
- Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
- Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
- Отсортируйте данные по общему объему продаж в порядке убывания.
- Выведите результаты на экран.


In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [7]:
conf = SparkConf().setAppName('les_5_hw')
sc = SparkContext(conf=conf)

spark = SparkSession.builder.getOrCreate()

csv_file_path = 'data/books.csv' 

25/04/30 00:07:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/30 00:07:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [9]:
df.show()

+--------------------+--------------------+------------------+------+------+
|               title|              author|             genre| sales|  year|
+--------------------+--------------------+------------------+------+------+
|                1984|     "George Orwell"| "Science Fiction"|5000.0|1949.0|
|The Lord of the R...|    "J.R.R. Tolkien"|         "Fantasy"|3000.0|1954.0|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"|4000.0|1960.0|
|The Catcher in th...|     "J.D. Salinger"|           "Novel"|2000.0|1951.0|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"|4500.0|1925.0|
+--------------------+--------------------+------------------+------+------+



In [10]:
df = df.withColumn("sales", df["sales"].cast('int'))
df = df.withColumn("year", df["year"].cast('int'))

In [11]:
# Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
df_filter = df.filter(df.sales > 3000)
df_filter.show()

+--------------------+--------------------+------------------+-----+----+
|               title|              author|             genre|sales|year|
+--------------------+--------------------+------------------+-----+----+
|                1984|     "George Orwell"| "Science Fiction"| 5000|1949|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"| 4000|1960|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"| 4500|1925|
+--------------------+--------------------+------------------+-----+----+



In [12]:
# Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
df_group_janr = df.groupby('genre').agg(F.sum('sales').alias('result_sales'))
df_group_janr.show()

+------------------+------------+
|             genre|result_sales|
+------------------+------------+
| "Southern Gothic"|        4000|
| "Science Fiction"|        5000|
|           "Novel"|        6500|
|         "Fantasy"|        3000|
+------------------+------------+



In [13]:
#Отсортируйте данные по общему объему продаж в порядке убывания.
df_order = df.orderBy(F.desc('sales'))
df_order.show()

+--------------------+--------------------+------------------+-----+----+
|               title|              author|             genre|sales|year|
+--------------------+--------------------+------------------+-----+----+
|                1984|     "George Orwell"| "Science Fiction"| 5000|1949|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"| 4500|1925|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"| 4000|1960|
|The Lord of the R...|    "J.R.R. Tolkien"|         "Fantasy"| 3000|1954|
|The Catcher in th...|     "J.D. Salinger"|           "Novel"| 2000|1951|
+--------------------+--------------------+------------------+-----+----+



In [14]:
sc.stop()